# Getting Started with cuDF
#### Originally By Yi Dong, Paul Hendricks

##### Modified by Chow Kah Mun
-------

While the world’s data doubles each year, CPU computing has hit a brick wall with the end of Moore’s law. For the same reasons, scientific computing and deep learning has turned to NVIDIA GPU acceleration, data analytics and machine learning where GPU acceleration is ideal. 

NVIDIA created RAPIDS – an open-source data analytics and machine learning acceleration platform that leverages GPUs to accelerate computations. RAPIDS is based on Python, has pandas-like and Scikit-Learn-like interfaces, is built on Apache Arrow in-memory data format, and can scale from 1 to multi-GPU to multi-nodes. RAPIDS integrates easily into the world’s most popular data science Python-based workflows. RAPIDS accelerates data science end-to-end – from data prep, to machine learning, to deep learning. And through Arrow, Spark users can easily move data into the RAPIDS platform for acceleration.

In this notebook, we will also show how to get started with GPU DataFrames using cuDF in RAPIDS.

**Table of Contents**

* Setup
* Loading data into a GPU DataFrame (GDF)
  * Loading data into a GDF
* Working with the GDF
  * Take a look at the columns and their data types
  * Slice the GDF
  * Modify data types
  * Manipulate data with a user-defined function (UDF)
  * Sort the data
  * Filter the data
  * One-hot encode categorical columns
  * Split the data into training and validation sets
  * Turn the GDFs into matrices
* Conclusion

## Setup

This notebook was tested using the `rapidsai/rapidsai:0.10-cuda10.0-runtime-ubuntu18.04` container from [DockerHub](https://hub.docker.com/r/rapidsai/rapidsai/tags) and run on the NVIDIA V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [1]:
!nvidia-smi

Thu Nov 14 09:29:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 430.50       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   36C    P0    37W / 250W |   1921MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:00:09.0 Off |                    0 |
| N/A   33C    P0    24W / 250W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
# initial setup
import cudf

## Loading data into a GPU DataFrame (GDF)

### What is IPUMS USA?

The Integrated Public Use Microdata Series (IPUMS USA) consists of more than fifty high-precision samples of the American population drawn from fifteen federal censuses and from the American Community Surveys. This is a small sample from the full dataset.


Loading data (json, csv, etc.) is easy.
For example, let's import some census data from a compressed CSV file on disk:

In [3]:
# read data from csv file into GPU dataframe
gdf = cudf.read_csv('./ipums_easy.csv')

Likewise to write a dataframe back to file we call the to_csv() method :

In [4]:
# write data to a csv file from GPU dataframe
gdf.to_csv('./output.csv')

And that's it! For the most part, working with GPU DataFrames will be the same as working with Pandas DataFrames. See the [cuDF documentation](https://docs.rapids.ai/api) for more information.

## Working with the GDF

### Take a look at the columns and their data types

In [5]:
# print the columns and their datatypes in this gdf
gdf.dtypes

RECTYPE          int64
YEAR             int64
DATANUM          int64
SERIAL           int64
NUMPREC          int64
SUBSAMP          int64
HHWT             int64
HHTYPE           int64
REPWT             int8
CLUSTER          int64
ADJUST         float64
CPI99          float64
REGION           int64
STATEICP         int64
STATEFIP         int64
COUNTY            int8
COUNTYFIPS        int8
METRO             int8
METAREA           int8
METAREAD          int8
MET2013           int8
MET2013ERR        int8
CITY              int8
CITYERR           int8
CITYPOP           int8
PUMA              int8
PUMARES2MIG       int8
STRATA           int64
PUMASUPR          int8
CONSPUMA          int8
                ...   
REPWTP51          int8
REPWTP52          int8
REPWTP53          int8
REPWTP54          int8
REPWTP55          int8
REPWTP56          int8
REPWTP57          int8
REPWTP58          int8
REPWTP59          int8
REPWTP60          int8
REPWTP61          int8
REPWTP62          int8
REPWTP63   

### Slice the GDF

Woah! This GDF has a lot of columns, let's make it more manageable...

In [6]:
# only select certain columns (and overwrite the gdf)
column_names = [
    'INCEARN', 'PERWT', 'ADJUST', 'STATEICP', 'ROOMS', 'BEDROOMS',
     'PHONE', 'VEHICLES', 'RACE', 'SEX', 'AGE', 'VETSTAT'
]
gdf = gdf.loc[:, column_names]

# show the first 5 records of each column
gdf.head(5)

,INCEARN,PERWT,ADJUST,STATEICP,ROOMS,BEDROOMS,PHONE,VEHICLES,RACE,SEX,AGE,VETSTAT
0,4000,618,1.018516,21,7,4,2,3,1,2,66,1
1,36700,684,1.018516,21,7,4,2,3,1,1,40,1
2,54000,618,1.018516,49,5,4,2,3,1,1,51,2
3,900,609,1.018516,49,5,4,2,3,1,2,48,1
4,2000,621,1.018516,49,5,4,2,3,1,1,19,1


### Modify data types

In [7]:
gdf.dtypes

INCEARN       int64
PERWT         int64
ADJUST      float64
STATEICP      int64
ROOMS         int64
BEDROOMS      int64
PHONE         int64
VEHICLES      int64
RACE          int64
SEX           int64
AGE           int64
VETSTAT       int64
dtype: object

Looks like `INCEARN` and `PERWT` are integers when they should be floats. Let's fix that...

In [8]:
import numpy as np

# convert the following two int64 columns to float64 data type
gdf['INCEARN'] = gdf['INCEARN'].astype(np.float64)
gdf['PERWT'] = gdf['PERWT'].astype(np.float64)

# take another look
gdf.dtypes

INCEARN     float64
PERWT       float64
ADJUST      float64
STATEICP      int64
ROOMS         int64
BEDROOMS      int64
PHONE         int64
VEHICLES      int64
RACE          int64
SEX           int64
AGE           int64
VETSTAT       int64
dtype: object

### Manipulate data with a user-defined function (UDF)

`INCEARN` is a column in our dataset that supposedly represents income earned; however, it does not truly represent the amount of income earned when adjusted for inflation. The `ADJUST` column represents the dollar inflation factor, which we can use to adjust `INCEARN` to the amount that the individual would have earned during the calender year. In our dataset, `ADJUST` is constant over all rows.

Below, we will define a simple function `adjust_incearn` that takes `INCEARN` and and multiplies it by a constant - in this case, the dollar inflation factor. We'll use the `applymap` method in our `cudf.dataframe.DataFrame` object to apply an element-wise function to transform the values in the Column.

In [9]:
# define a function to adjust the incearn column
# so it more accurately represents income earned
adjust = gdf['ADJUST'][0]   # take constant from first row
print('adjustment factor: {}'.format(adjust))

print('mean income: {}'.format(gdf['INCEARN'].mean()))

def adjust_incearn(incearn):
    return adjust * incearn;

# apply it to the 'INCEARN' column, row by row
gdf['INCEARN'] = gdf['INCEARN'].applymap(adjust_incearn)

# compute the mean
print('mean adjusted income: {}'.format(gdf['INCEARN'].mean()))

adjustment factor: 1.018516
mean income: 18298.2888
mean adjusted income: 18637.0999154208


In [10]:
# drop the ADJUST column
gdf.drop_column('ADJUST')

### Sort the data

Next, let's sort out data to do some light exploration.

In [11]:
# sort the gdf by the INCEARN column
gdf = gdf.sort_values(by='INCEARN', ascending=False)

# reset the index so we can use loc slicing later
gdf = gdf.reset_index()
gdf

,index,INCEARN,PERWT,STATEICP,ROOMS,BEDROOMS,PHONE,VEHICLES,RACE,SEX,AGE,VETSTAT
0,9108,604998.504000,632.0,13,9,6,2,1,1,2,67,1
1,5835,571387.476000,213.0,98,9,6,2,4,1,2,45,1
2,7331,526572.772000,720.0,56,9,4,2,2,1,1,68,2
3,3236,500091.356000,627.0,24,5,4,2,2,1,1,42,1
4,5989,405369.368000,551.0,23,7,4,2,2,1,1,61,1
5,2909,403332.336000,544.0,1,9,6,2,2,1,1,45,1
6,491,397221.240000,657.0,13,9,6,2,2,1,1,68,1
7,5474,397221.240000,627.0,13,4,2,2,9,1,2,47,1
8,8051,397221.240000,819.0,13,7,4,2,3,1,1,35,1
9,1358,376850.920000,630.0,47,9,5,2,2,1,1,52,1


Looks like we have some negative income values. Let's filter those out...

### Filter the data

We'll use the `query` method to filter our dataset. The `query` method takes as argument a boolean expression very similar to the `query` method for the `pandas.DataFrame` class. However, the `cudf.dataframe.DataFrame` implementation uses Numba to compile a GPU kernel. 

For more information on the syntax for arguments into `query`, see the Pandas documentation: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html

In [12]:
# how many records do we have?
print("{} = Original # of records".format(len(gdf)))

# filter out
gdf = gdf.query('INCEARN >= 0')

# how many records do we have left?
print("{} = New # of records".format(len(gdf)))

# sanity check...
gdf

10000 = Original # of records
9985 = New # of records


,index,INCEARN,PERWT,STATEICP,ROOMS,BEDROOMS,PHONE,VEHICLES,RACE,SEX,AGE,VETSTAT
0,9108,604998.504,632.0,13,9,6,2,1,1,2,67,1
1,5835,571387.476,213.0,98,9,6,2,4,1,2,45,1
2,7331,526572.772,720.0,56,9,4,2,2,1,1,68,2
3,3236,500091.356,627.0,24,5,4,2,2,1,1,42,1
4,5989,405369.368,551.0,23,7,4,2,2,1,1,61,1
5,2909,403332.336,544.0,1,9,6,2,2,1,1,45,1
6,491,397221.240,657.0,13,9,6,2,2,1,1,68,1
7,5474,397221.240,627.0,13,4,2,2,9,1,2,47,1
8,8051,397221.240,819.0,13,7,4,2,3,1,1,35,1
9,1358,376850.920,630.0,47,9,5,2,2,1,1,52,1


### Groupbys

A useful operation when working with datasets is to group the data using a specific key and aggregate the values mapping to those keys. For example, we might want to aggregate by State and Race data and sum up the earning income of each demographic.

cuDF allows us to perform such an operation using the `groupby` method. This will create an object of type `cudf.DataFrame.groupby` that we can operate on using aggregation functions such as `sum`, `min`, `max` or complex aggregation functions defined by the user.

We can also specify multiple columns to group on by passing a list of column names to the `groupby` method.

https://usa.ipums.org/usa-action/variables/US2000A_1022#description_section
https://usa.ipums.org/usa-action/variables/STATEICP#codes_section

* 1	White alone
* 2	Black or African American alone
* 3	American Indian alone
* 4	Alaska Native alone
* 5	American Indian and Alaska Native tribes specified, and American Indian or Alaska Native, not specified, and no other races
* 6	Asian alone
* 7	Native Hawaiian and Other Pacific Islander alone
* 8	Some other race alone
* 9	Two or more major race groups


Asians has highest income in California State in the year 2000.

In [13]:
# only select columns we are interested in for example the state and race 
column_names = [
    'INCEARN', 'STATEICP', 'RACE'
]
egdf = gdf.loc[:, column_names]

# perform a group_by based on the State and Race
grouped_df = egdf.groupby(['STATEICP','RACE'])

# and then aggregate by sum of income earned
aggregation = grouped_df.sum()
aggregation

INCEARN
STATEICP RACE              
1        1     2.362754e+06
         2     5.968504e+04
         6     1.818051e+05
         7     6.925909e+04
         8     0.000000e+00
2        1     9.291921e+05
         2     6.111096e+04
         3     1.222219e+04
3        1     3.818651e+06
         2     4.573137e+04
         3     1.262960e+05
         4     3.707398e+04
         6     2.546290e+05
         7     1.018516e+04
         8     2.342587e+04
4        1     1.355330e+06
         7     2.138884e+04
5        1     2.182965e+06
         4     1.324071e+05
         8     1.425922e+04
6        1     1.364455e+06
11       1     1.460012e+06
         2     1.995273e+05
         3     0.000000e+00
12       1     5.215138e+06
         2     7.845629e+05
         4     5.092580e+04
         5     0.000000e+00
         6     5.758689e+05
         7     1.117312e+05
...                     ...
71       6     1.455001e+06
         7     9.294977e+05
         8     2.864067e+05
         9     6.345355e+04
72       1     2.802550e+06
         2     6.111096e+03
         8     7.129612e+04
         9     0.000000e+00
73       1     3.294186e+06
         2     1.016479e+05
         5     4.471285e+03
         6     5.805541e+04
         7     8.901830e+04
         8     1.680551e+05
81       1     1.437737e+06
         2     1.481941e+05
         3     5.041654e+04
         6     4.888877e+04
         8     3.666658e+04
82       1     7.533973e+05
         2     8.759238e+02
         4     3.978323e+05
         5     4.112768e+05
         6     6.559243e+04
         7     5.856467e+04
         8     6.925909e+03
         9     2.322216e+04
98       1     1.535413e+06
         2     4.090360e+05
         8     4.481470e+04

[239 rows x 1 columns]

### One-hot encode categorical columns

Next, let's prepare our categorical columns. Machine learning models won't take strings as inputs, so we need to go to each column and convert its string representations to a numerical representation. The most common way to convert a Column with `n` elements and `k` unique categories to a numerical representation is to create a matrix of shape `n` by `k` and impute a 1 in cell `(i, j)` if the `ith` element is of category `j` and 0 otherwise, where $j \in k$. This is known as one-hot encoding.

In [14]:
# define the categorical columns
cat_cols = set(['STATEICP', 'RACE', 'SEX', 'VETSTAT'])
# store the unique values for each category column
uniques = {}

# iterate through each categorical column and one-hot
# encode it using the unique values it has
# for k in cat_cols:
#     uniques[k] = gdf[k].unique_k(k=1000)
#     cats = uniques[k][1:]  # drop first
#     gdf = gdf.one_hot_encoding(k, prefix=k, cats=cats)
#     #del gdf[k]

# iterate through each categorical column and one-hot
# encode it using the unique values it has
for k in cat_cols:
    uniques[k] = gdf[k].unique()
    gdf = gdf.one_hot_encoding(k, prefix=k, cats=uniques[k])
    del gdf[k]
    
# we should see many more columns since the categorical
# columns will get expanded due to one-hot encoding
gdf.dtypes

index            int64
INCEARN        float64
PERWT          float64
ROOMS            int64
BEDROOMS         int64
PHONE            int64
VEHICLES         int64
AGE              int64
SEX_1          float64
SEX_2          float64
RACE_1         float64
RACE_2         float64
RACE_3         float64
RACE_4         float64
RACE_5         float64
RACE_6         float64
RACE_7         float64
RACE_8         float64
RACE_9         float64
STATEICP_1     float64
STATEICP_2     float64
STATEICP_3     float64
STATEICP_4     float64
STATEICP_5     float64
STATEICP_6     float64
STATEICP_11    float64
STATEICP_12    float64
STATEICP_13    float64
STATEICP_14    float64
STATEICP_21    float64
                ...   
STATEICP_42    float64
STATEICP_43    float64
STATEICP_44    float64
STATEICP_45    float64
STATEICP_46    float64
STATEICP_47    float64
STATEICP_48    float64
STATEICP_49    float64
STATEICP_51    float64
STATEICP_52    float64
STATEICP_53    float64
STATEICP_54    float64
STATEICP_56

### Split the data into training and validation sets

Next, let's split out data into an 80% train dataset and a 20% validation dataset.

In [15]:
# enforce float64 data type on ALL columns
for k in gdf.columns:
    gdf[k] = gdf[k].astype(np.float64)

# set the fractions for training and validation
fractions = {
    "train": 0.8,
    "valid": 0.2
}

# validation splitpoint
splitpoint = int(len(gdf) * fractions["train"])
print('splitpoint: {} of {} is {}'.format(fractions["train"], len(gdf), splitpoint))

# break the gdf up into training and validation sets
gdfs = {
    "train": gdf.loc[:splitpoint],
    "valid": gdf.loc[splitpoint:]
}
print('gdfs["train"] has {} rows'.format(len(gdfs["train"])))
print('gdfs["valid"] has {} rows'.format(len(gdfs["valid"])))

splitpoint: 0.8 of 9985 is 7988
gdfs["train"] has 7989 rows
gdfs["valid"] has 1997 rows


### Turn the GDFs into matrices

Lastly, we want to convert our GPU DataFrame to a GPU Matrix for usage as input to other machine learning libraries such as cuML and XGBoost. We can use the `as_gpu_matrix` method to facillitate this conversion.

In [16]:
gdf.columns

Index(['index', 'INCEARN', 'PERWT', 'ROOMS', 'BEDROOMS', 'PHONE', 'VEHICLES',
       'AGE', 'SEX_1', 'SEX_2', 'RACE_1', 'RACE_2', 'RACE_3', 'RACE_4',
       'RACE_5', 'RACE_6', 'RACE_7', 'RACE_8', 'RACE_9', 'STATEICP_1',
       'STATEICP_2', 'STATEICP_3', 'STATEICP_4', 'STATEICP_5', 'STATEICP_6',
       'STATEICP_11', 'STATEICP_12', 'STATEICP_13', 'STATEICP_14',
       'STATEICP_21', 'STATEICP_22', 'STATEICP_23', 'STATEICP_24',
       'STATEICP_25', 'STATEICP_31', 'STATEICP_32', 'STATEICP_33',
       'STATEICP_34', 'STATEICP_35', 'STATEICP_36', 'STATEICP_37',
       'STATEICP_40', 'STATEICP_41', 'STATEICP_42', 'STATEICP_43',
       'STATEICP_44', 'STATEICP_45', 'STATEICP_46', 'STATEICP_47',
       'STATEICP_48', 'STATEICP_49', 'STATEICP_51', 'STATEICP_52',
       'STATEICP_53', 'STATEICP_54', 'STATEICP_56', 'STATEICP_61',
       'STATEICP_62', 'STATEICP_63', 'STATEICP_64', 'STATEICP_65',
       'STATEICP_66', 'STATEICP_67', 'STATEICP_68', 'STATEICP_71',
       'STATEICP_72', 'STATEICP_

In [17]:
# produce gpu matrices (to input to ML libraries, etc.)
# this step should not be necessary in the near future
# (should be able to use gdf as input)
matrices = {
    "train": {
        "x": gdfs["train"].as_gpu_matrix(columns=gdf.columns[1:]),
        "y": gdfs["train"].as_gpu_matrix(columns=[gdf.columns[0]])
    },
    "valid": {
        "x": gdfs["valid"].as_gpu_matrix(columns=gdf.columns[1:]),
        "y": gdfs["valid"].as_gpu_matrix(columns=[gdf.columns[0]])
    }
}

# check the matrix shapes (sanity check)
print('matrices["train"]["x"] shape:', matrices["train"]["x"].shape)
print('matrices["train"]["y"] shape:', matrices["train"]["y"].shape)
print('matrices["valid"]["x"] shape:', matrices["valid"]["x"].shape)
print('matrices["valid"]["y"] shape:', matrices["valid"]["y"].shape)

matrices["train"]["x"] shape: (7989, 72)
matrices["train"]["y"] shape: (7989, 1)
matrices["valid"]["x"] shape: (1997, 72)
matrices["valid"]["y"] shape: (1997, 1)


## Conclusion

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)

Credits go to Yi Dong and Paul Hendricks for authoring and creating the base notebook
* https://github.com/rapidsai/notebooks-contrib/blob/master/getting_started_notebooks/basics/Getting_Started_with_cuDF.ipynb